# Information

You can run the entire script by press **CTRL+F9**.

For more helpful scripts, check out the quick commands.
*   [Robohood | Quick commands](https://colab.research.google.com/drive1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5?usp=sharing)
*  [Cancel all pending orders](https://colab.research.google.com/drive/1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5#scrollTo=ElxYAXz3LWR5&line=1&uniqifier=1)
*  [Close all positions](https://colab.research.google.com/drive/1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5#scrollTo=LD2LSDnqOlFe&line=1&uniqifier=1)

# Application code
Only needs to be run once per session.

In [1]:
%pip install robin_stocks

Note: you may need to restart the kernel to use updated packages.


In [2]:
from robin_stocks import robinhood as r
import itertools
from datetime import datetime
import getpass
import sys
import time
from datetime import datetime, timedelta
from dateutil.parser import parse
from pytz import timezone


## Funcitons

In [3]:
def get_spread_cost(front_leg_option, back_leg_option, params):
  spread_cost = -1
  front_price = front_leg_option.get(params[0])
  back_price = back_leg_option.get(params[1])

  if (back_price and front_price):
    spread_cost = round(float(back_price) - float(front_price), 2)

  return spread_cost

In [4]:
def get_min_order_quantity(spread):
  (front_leg_option, back_leg_option, spread_cost, profitability) = spread

  back_size = back_leg_option.get("ask_size")
  front_size = front_leg_option.get("bid_size")

  if (front_size and back_size):
    return min(int(front_size), int(back_size))
  else:
    return 0

In [5]:
def get_chance_of_profit(front_leg_option, back_leg_option):
    chance_of_profit_short = front_leg_option.get("chance_of_profit_short")
    chance_of_profit_long = back_leg_option.get("chance_of_profit_long")
    if (chance_of_profit_short and chance_of_profit_long):
        return float(chance_of_profit_short) + float(chance_of_profit_long)
    return 0

In [6]:
def find_calendar_spreads(options, params):
  # Create a create new calendar spread for every combination of expiration dates where the short front leg expiration_date is less than the long back leg of the spread
  sorted_options = sorted(options, key=lambda x: (float(x['strike_price'])))
  options_grouped_by_strike = itertools.groupby(sorted_options, lambda x : x['strike_price'])
  today = datetime.today()

  #  using the front_leg_option and back_leg_option as the front and back legs of the trade, create a debit calendar spread
  #  with a cost of zero
  calendar_spreads = []
  for strike, options in options_grouped_by_strike:
    for (front_leg_option, back_leg_option) in itertools.combinations(options,2):
      front_expiration_date = datetime.strptime(front_leg_option["expiration_date"],'%Y-%m-%d')
      back_expiration_date = datetime.strptime(back_leg_option["expiration_date"],'%Y-%m-%d')
      optionTypeIsSame = front_leg_option["type"] == back_leg_option["type"]

      if (optionTypeIsSame and (front_expiration_date > today) and (front_expiration_date < back_expiration_date)):
        spread_cost = get_spread_cost(front_leg_option, back_leg_option, params)
        profitability = get_chance_of_profit(front_leg_option, back_leg_option)
        spread = (front_leg_option, back_leg_option,
                  spread_cost, profitability)

        calendar_spreads.append(spread)

  return calendar_spreads

In [7]:
def get_current_price_of_symbol(symbol="SPY"):
  current_price = round(float(r.stocks.get_latest_price(symbol)[0]))
  print("{} is currently trading at: ${}".format(symbol, current_price))
  return current_price 

In [8]:
def scan_for_options(symbol, optionType, min_strike_offset, max_strike_offset):
  if(optionType == 'both'):
    optionType = ""

  current_price = get_current_price_of_symbol(symbol)
  min_strike = current_price - min_strike_offset;
  max_strike = current_price + max_strike_offset;

  # Get the list of options available and sort by strike_price and expiration_date
  options = r.options.find_tradable_options(symbol, optionType=optionType, info=None)
  options = list(filter(lambda x: (float(x["strike_price"]) > min_strike) and (float(x["strike_price"]) < max_strike), options))
  return options

In [9]:
def get_date_time(date_time_str, zone='US/Eastern'):
    # converting the UTC input to a datetime object
  utc_datetime = parse(date_time_str)

  # converting the UTC datetime object to Eastern Time
  est_datetime = utc_datetime.astimezone(tz=timezone(zone))
  # returning the EST datetime as a string in year-month-day hour:min am/pm format
  return est_datetime.strftime("%Y-%m-%d %I:%M %p")
  
def print_option(item):
  zone = 'US/Eastern'
  option_type = item.get('type')
  expiration_date = item.get('expiration_date')
  strike_price = item.get('strike_price')
  last_updated = get_date_time(item['updated_at'], zone)

  if(strike_price):
    strike_price = round(float(strike_price), 2) 
  last_price = item['last_trade_price']
  if(last_price):
    last_price = round(float(last_price), 2)

  print(
      f"{option_type} | Strike: ${strike_price} | Expiration date: {expiration_date} | Last price: ${last_price} | Last updated: {last_updated} ({zone})")


In [10]:
def print_spread(spread):
  (front_leg_option, back_leg_option, spread_cost, profitability) = spread
  strike = float(front_leg_option.get("strike_price"))
  print("strike {} | short {} @ {} | long {} @ {} | spread: ${} | profitability: {}".format(
      strike,
      front_leg_option.get("type"),
      front_leg_option.get("expiration_date"),
      back_leg_option.get("type"),
      back_leg_option.get("expiration_date"),
      spread_cost,
      profitability))

In [11]:
def print_spreads(spreads):
  for spread in spreads:
    print_spread(spread)

In [12]:
from time import sleep
def order_calendar_spread(spread, price=0.00, quantity=1, timeInForce='gfd', max_attempts=3, sleep_time=1):
  (front_leg_option, back_leg_option, spread_cost, profitability) = spread
  symbol = front_leg_option["symbol"]
  params = [
            {
            'expirationDate': front_leg_option['expiration_date'],
            'strike': front_leg_option['strike_price'],
            'optionType': front_leg_option['type'],
            'quantity': '1',
            'effect': 'open',
            'action': 'sell',
          },
          {
            'expirationDate': back_leg_option['expiration_date'],
            'strike': back_leg_option['strike_price'],
            'optionType': back_leg_option['type'],
            'quantity': '1',
            'effect': 'open',
            'action': 'buy',
          },
  ]
  
  print("buying {} @ ${}".format(quantity, price))

  order = r.orders.order_option_spread(direction='debit', price=price, symbol=symbol,
                                       quantity=quantity, spread=params, timeInForce=timeInForce)
  if(order):
    if (order.get("state")):
      print_spread(spread)
      return order

    if (order.get("detail")):
        print(order.get("detail"))
        return order
  else:
    attempts = 0
    while attempts < max_attempts:
      attempts += 1
      print("Something failed. Auto retry attempt {} of {}".format(attempts, max_attempts))
      order = r.orders.order_option_spread(direction='debit', price=price, symbol=symbol,
                                          quantity=quantity, spread=params, timeInForce=timeInForce)
      sleep(sleep_time)

      if attempts == max_attempts:
          print("max number of tries exceeded. Order failed because ")
          print(order.get("detail"))

  return order

In [13]:
def place_calendar_spreads(spreads, price_type, quantity_type, timeInForce, max_quantity=250, price=0.00, quantity=1):
  results = []
  if(price_type == "set_for_all"):
    price = float(input("Please enter the PRICE for each order: ") or 0.00)

  if(quantity_type == "set_for_all"):
    quantity = int(input("Please enter the QUANTITY for each order: ") or 0)

  if (quantity_type == "use_max_available_spreads"):
    user_max_quantity = int(input("Please enter the MAX_QUANTITY for each order: ") or max_quantity)

  for spread in spreads:
    (front_leg_option, back_leg_option, spread_cost, profitability) = spread

    if(price_type == "set_each"):
      price = float(input("price (limit): ") or 0.00)

    if (price_type == "use_spread_cost"):
      price = spread_cost

    if(quantity_type == "set_each"):
      quantity = int(input("order size (quantity): ") or 1)

    if (quantity_type == "use_max_available_spreads"):
      quantity = get_min_order_quantity(spread)
      if ((quantity >= user_max_quantity)):
        quantity = user_max_quantity

    if ((quantity >= max_quantity)):
      quantity = max_quantity

    result = order_calendar_spread(spread=spread, price=price, quantity=quantity, timeInForce=timeInForce)
    results.append(result)
  return results

In [14]:
import threading

def market_worker(item):
  marketData = r.options.get_option_market_data_by_id(item['id'])
  if marketData:
      item.update(marketData[0])
      print_option(item)


def update_option_market_data(options):
  for item in options:
    thread = threading.Thread(target=market_worker(item)).start()

In [25]:
def spread_filter(spread, max_spread_cost, min_volume, min_avalible_quantity, min_profitability,  min_front_short_date, max_front_short_date, min_back_long_date, max_back_long_date):
    (front_leg_option, back_leg_option, spread_cost, profitability) = spread
    front_volume = front_leg_option.get("volume")
    back_volume = back_leg_option.get("volume")
    is_in_volume_range=False

    is_less_than_max_cost = (spread_cost <= max_spread_cost)
    if (front_volume and back_volume):
        is_in_volume_range = (int(front_volume) >= min_volume) and (int(back_volume) >= min_volume)
    is_greater_than_min_avalible_quantity = get_min_order_quantity(spread) >= min_avalible_quantity
    is_greater_than_profitability = profitability >= min_profitability

    front_date = parse(front_leg_option["expiration_date"])
    back_date = parse(back_leg_option["expiration_date"])

    is_greater_than_min_front_short_date = front_date >= min_front_short_date
    is_less_than_max_front_short_date = front_date <= max_front_short_date
    is_greater_than_min_back_long_date = back_date >= min_back_long_date
    is_less_than_max_back_long_date = back_date <= max_back_long_date

    return (is_less_than_max_cost and is_in_volume_range and is_greater_than_min_avalible_quantity and is_greater_than_profitability and is_greater_than_min_front_short_date and is_less_than_max_front_short_date and is_greater_than_min_back_long_date and is_less_than_max_back_long_date)


In [16]:
def filter_calendar_spreads(spreads, max_spread_cost, min_volume, min_avalible_quantity, min_chance_of_profitability, min_front_short_date, max_front_short_date, min_back_long_date, max_back_long_date):

  filtered_spreads = list(
      filter(lambda spread: spread_filter(spread, max_spread_cost, min_volume, min_avalible_quantity, min_chance_of_profitability, min_front_short_date, max_front_short_date, min_back_long_date, max_back_long_date), spreads))

  filtered_spreads = sorted(
      filtered_spreads, key=lambda spread: ((spread[2]), -spread[3]))
  return filtered_spreads

# Login

In [17]:
#@title #Authenticate { vertical-output: true, display-mode: "form" }
username = "forrest.surprenant@gmail.com" #@param {type:"string"}
password = ""
login = r.login(username, password)
print(login.get("detail"))

logged in using authentication in robinhood.pickle


# Scan for options

> Inputs needed: offset and max offset. By default, the scanner will analize $10 worth of different strikes. 5 above and 5 below.

* **symbol**: example "spy"
* **min strike offset**: how far below the current strike you want to look. The default is 5 dollars.
* **max strike offset**: how far above the current strike you want to look. The 
default is 5 dollars.

In [26]:
#@title Enter option criteria { vertical-output: true, display-mode: "form" }
# Create a script that places a debit calendar spread for each tradeable option of a given symbol. The cost of the spread needs to be zero.
symbol = "BBBY"  # @param {type:"string"}
optionType = "both"  # @param ["call", "put", "both"]
min_strike_offset = 5 # @param {type:"integer"}
max_strike_offset = 5  # @param {type:"integer"}

options = scan_for_options(symbol, optionType, min_strike_offset, max_strike_offset)
print("Base on user criteria, you found {} to analize!".format(len(options)))

BBBY is currently trading at: $3
Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Base on user criteria, you found 298 to analize!


## Get option market data


> This gets the latest market information for each option found. Data such as ask_price and bid_price are changing all the time. It's a good idea to refresh this often.

In [27]:
update_option_market_data(options)

call | Strike: $5.0 | Expiration date: 2023-01-27 | Last price: $0.67 | Last updated: 2023-01-13 03:59 PM (US/Eastern)
put | Strike: $1.5 | Expiration date: 2023-02-03 | Last price: $0.29 | Last updated: 2023-01-13 03:59 PM (US/Eastern)
call | Strike: $7.5 | Expiration date: 2023-02-10 | Last price: $0.86 | Last updated: 2023-01-13 03:59 PM (US/Eastern)
put | Strike: $4.5 | Expiration date: 2023-02-03 | Last price: $2.22 | Last updated: 2023-01-13 03:59 PM (US/Eastern)
put | Strike: $2.0 | Expiration date: 2025-01-17 | Last price: $1.57 | Last updated: 2023-01-13 03:53 PM (US/Eastern)
call | Strike: $5.5 | Expiration date: 2023-03-03 | Last price: $0.86 | Last updated: 2023-01-13 03:59 PM (US/Eastern)
call | Strike: $3.0 | Expiration date: 2023-03-17 | Last price: $1.52 | Last updated: 2023-01-13 03:59 PM (US/Eastern)
call | Strike: $6.0 | Expiration date: 2023-02-24 | Last price: $0.91 | Last updated: 2023-01-13 03:59 PM (US/Eastern)
put | Strike: $3.0 | Expiration date: 2023-02-03 | 

# Filter calendar spreads (you should live that this block)


**short_price_key** is the price you wish to **sell** the front leg of the calendar.

**long_price_key** is the price you wish to **buy** the back leg of the calendar.

**max_spread_cost** filter your results by the cost for each spread. The spread cost is the price of your long_price - short_price. 

Example: ask_price - bid_price = spread_cost. The default is $0.50.

**min_avalible_quantity** filter your results by the number of bidders for your front short leg and the number of sellers for your back long leg.

**min_chance_of_profitability** is the sum of chance of profit short + chance of long. Thorectically, a value of 1 or greater means you have the thorectical edge.

In [31]:
#@title Filter based on spread cost { vertical-output: true, display-mode: "form" }
max_spread_cost = 4  # @param {type:"number"}
min_volume = 0  # @param {type:"integer"}
min_avalible_quantity = 0  # @param {type:"integer"}
min_chance_of_profitability = 1 # @param {type:"number"}
short_price_key = "ask_price" # @param ["mark_price", "last_trade_price", "bid_price", "ask_price", "low_price", "high_price", "high_fill_rate_buy_price","high_fill_rate_sell_price", "low_fill_rate_buy_price", "low_fill_rate_sell_price", "adjusted_mark_price", "adjusted_mark_price_round_down"]
long_price_key = "bid_price" # @param ["mark_price", "last_trade_price", "bid_price", "ask_price", "low_price", "high_price", "high_fill_rate_buy_price","high_fill_rate_sell_price", "low_fill_rate_buy_price", "low_fill_rate_sell_price", "adjusted_mark_price", "adjusted_mark_price_round_down"]
min_front_short_date = "2023-01-17" #@param {type:"date"}
max_front_short_date = "2023-01-30" #@param {type:"date"}
min_back_long_date = "2023-01-17" #@param {type:"date"}
max_back_long_date =  "2023-01-30" #@param {type:"date"}


min_front_short_date = parse(min_front_short_date)
max_front_short_date = parse(max_front_short_date)
min_back_long_date = parse(min_back_long_date)
max_back_long_date = parse(max_back_long_date)

params = [short_price_key, long_price_key]

calendar_spreads = find_calendar_spreads(options, params)
filtered_spreads = filter_calendar_spreads(
    calendar_spreads, max_spread_cost, min_volume, min_avalible_quantity, min_chance_of_profitability, min_front_short_date, max_front_short_date, min_back_long_date, max_back_long_date)
print("{} spreads found.".format(len(filtered_spreads)))
print_spreads(filtered_spreads)

12 spreads found.
strike 1.5 | short call @ 2023-01-20 | long call @ 2023-01-27 | spread: $-0.65 | profitability: 1.049233
strike 1.0 | short put @ 2023-01-20 | long put @ 2023-01-27 | spread: $0.01 | profitability: 1.036742
strike 1.5 | short put @ 2023-01-20 | long put @ 2023-01-27 | spread: $0.06 | profitability: 1.079738
strike 7.0 | short call @ 2023-01-20 | long call @ 2023-01-27 | spread: $0.09 | profitability: 1.010513
strike 5.5 | short put @ 2023-01-20 | long put @ 2023-01-27 | spread: $0.18 | profitability: 1.02651
strike 5.5 | short call @ 2023-01-20 | long call @ 2023-01-27 | spread: $0.18 | profitability: 1.008077
strike 6.0 | short call @ 2023-01-20 | long call @ 2023-01-27 | spread: $0.19 | profitability: 1.0105790000000001
strike 7.5 | short put @ 2023-01-20 | long put @ 2023-01-27 | spread: $0.2 | profitability: 1.033606
strike 2.5 | short put @ 2023-01-20 | long put @ 2023-01-27 | spread: $0.26 | profitability: 1.106436
strike 4.5 | short put @ 2023-01-20 | long put 

# Pick your top orders. 
Create calendar spread orders (When you find something you like)

Select the number of order you want to place. Note: the max number of orders you can create in robinhood is 11.

In [21]:
#@title How many orders do you want to place? { run: "auto", vertical-output: true, display-mode: "form" }
max_order_size = 11 #@param {type:"integer"}
top_filtered_spreads = filtered_spreads[:max_order_size] # robinhood only allows 11 orders
print_spreads(top_filtered_spreads)

In [22]:
#@title Place calendar orders based on the following parameters: { vertical-output: true, display-mode: "form" }
ask_for_confirmation = "False" # @param bool["True", "False"]

price_type = "set_for_all" # @param ["set_for_all", "set_each", "use_spread_cost"]
quantity_type = "use_max_available_spreads" # @param ["set_for_all", "set_each", "use_max_available_spreads"]
timeInForce='gfd'  #@param ["gfd", "gtc"] 

place_trades_answer = input("Place your trades you just found? Enter 'y' or 'n' to continue: " or 'n')

if (ask_for_confirmation != "True" or place_trades_answer.casefold() == "y"):
    place_calendar_spreads(top_filtered_spreads, price_type, quantity_type, timeInForce)
    print("Done.")
else:
    print("No trades were placed.")

Done.


In [23]:
#@title Cancel all pending orders
ask_for_confirmation = "False" # @param bool["True", "False"]

def cancel_all_pending_orders():
  if (ask_for_confirmation != "True" or input("Cancel all trades? Enter 'y' or 'n' to continue: " or 'n').casefold() == "y"):
    #Get all pending orders
    pending_orders = r.orders.get_all_open_option_orders()

    #Loop through orders and cancel each one
    for order in pending_orders:
        print("Canceling order: ", order["id"])
        r.cancel_option_order(order["id"])
  else:
    print("No trades were canceled.")

cancel_all_pending_orders()

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Canceling order:  63c60763-9e8d-4e64-a86b-162cb914f3c0
Canceling order:  63c603e9-2070-4693-b493-b2519493b4bc
Canceling order:  63c603e4-536d-4ce9-857a-befdad8d11a7
Canceling order:  63c603dd-9a7e-4946-b72e-6bf0a1bc234e
Canceling order:  63c5a572-9b90-41a6-8ae2-7c9f72e93251
Canceling order:  63c5a570-9678-4cf5-875f-57adb7eb1a6d
Canceling order:  63c5a56f-e9a7-486f-98ba-2681234d74f3
Canceling order:  63c5a56d-6db0-4254-bce0-186e2e3de81c
Canceling order:  63c5a56c-b99f-409d-be59-c1d3c751fd84
Canceling